In [3]:
# import generator from '../data/generator.py'
import sys
import os
from generator import RandomGraphDataset

In [9]:
import torch
from torch import nn
from encoder import Encoder
from decoder import Decoder
from mpnn import MPNN
from torch.functional import F

class Network(nn.Module):
    def __init__(self, latent_dim=128):
        super(Network, self).__init__()
        self.encoder = Encoder(2, latent_dim)
        self.processor = MPNN(latent_dim*2, latent_dim)
        self.decoder = Decoder(latent_dim, 1)

    def forward(self, batch, max_iter=10):
        input = torch.stack((batch.pos, batch.s), dim=1).float()
        h = torch.zeros(input.size(0), 128) # hidden state from the processor
        hints_edges = batch.edges_h[1:] # hints if an edge was passed or not
        true_output = batch.edges # true_output for all the edges if they were passed or not at the end.
        max_iter = hints_edges.size(0)

        hints_reach = batch.reach_h[1:] # hints from the reachability
        
        predictions_edges = torch.zeros(max_iter, batch.edges.size(0))

        for i in range(max_iter):
            threshold = 0.5

            # NETWORK START
            z = self.encoder(input) # the encoded input
            processor_input = torch.cat([z, h], dim=1) # the input to the processor
            h = self.processor(processor_input, batch.edge_index.long()) # the output of the processor
            decoder_input = torch.cat((h[batch.edge_index[0]], h[batch.edge_index[1]]), dim=1)
            alpha = self.decoder(decoder_input).view(batch.edges.size(0))
            # NETWORK END

            # predictions for the edges
            predictions_edges[i] = alpha.view(batch.edges.size(0))
            # update the input with the new state
            predictions_reach = self.calculate_reach(batch, alpha, threshold=0.4)
            
            input = torch.stack((batch.pos, predictions_reach), dim=1).float()

        # predictions for reach 

        # predictions for the parents
        predictions_parents = self.get_parent_nodes(batch.edge_index, alpha, batch.s, threshold=0.)
        
        loss_edges = self.calculate_loss(hints_edges, predictions_edges, true_output)
        loss_reach = self.calculate_reach_loss(predictions_reach, batch.reach_h[-1])
        loss_parents = self.calculate_parents_loss(predictions_parents, batch.pi)
        
        return loss_edges, loss_reach, loss_parents
        
    def calculate_loss(self, hints, predictions, true_output):
        if len(predictions) == 0:
            return torch.tensor(0.0), torch.tensor(0.0)  # Return zero loss if predictions is empty
        loss_x = F.binary_cross_entropy(predictions[-1], true_output.type(torch.float))
        loss_h = 0
        for i in range(hints.size(0)):
            loss_h += F.binary_cross_entropy(predictions[i], hints[i].type(torch.float))
        return loss_x, loss_h
    
    def calculate_reach_loss(self, predictions, true_output):
        return F.binary_cross_entropy(predictions, true_output.type(torch.float))
    

    def calculate_reach(self, graph, alpha, threshold=0.8):
        """
        Calculate reachability values for each node from the alpha values

        Args:
        - graph: Graph object containing graph data
        - alpha: PyTorch tensor containing alpha values
        - threshold: Threshold value for reachability

        Returns:
        - y: PyTorch tensor containing reachability values for each node
        """
        y = torch.zeros((len(graph.s)))
        for node_index in range(len(graph.s)):
            # Check if there are any edges connected to the current node
            connected_edges = torch.logical_or(graph.edge_index[0] == node_index, graph.edge_index[1] == node_index)
            if torch.any(connected_edges):
                alpha_max_proba = alpha[connected_edges].max()
                if alpha_max_proba.item() >= threshold:
                    y[node_index] = 1
        return y


    def get_parent_nodes(self, edge_index, alpha, s, threshold=0.8):
        num_nodes = len(s)
        parent_nodes = torch.arange(num_nodes)  # Initialize parent nodes with their own index

        for node in range(num_nodes):
            # Get all edges that point to the current node
            incoming_edges = (edge_index[1] == node).nonzero(as_tuple=False).squeeze()
            
            # Check if there are any edges pointing to the current node
            if incoming_edges.numel() != 0:
                # Filter alpha based on threshold
                filtered_edges = incoming_edges[alpha[incoming_edges] >= threshold]
                
                # Check if there are any edges left after filtering
                if filtered_edges.numel() != 0:
                    # Get the index of the edge with the highest alpha after filtering
                    max_alpha_index = torch.argmax(alpha[filtered_edges])
                    # Get the parent node
                    parent_nodes[node] = edge_index[0, filtered_edges[max_alpha_index]].item()

        return parent_nodes

    def calculate_parents_loss(self, predictions_parents, final_parents):
        if len(predictions_parents) == 0 or len(final_parents) == 0:
            return torch.tensor(1.0) 
        return 1 - torch.mean(torch.eq(predictions_parents, final_parents).float())

In [10]:
def train(model, train_dataset, validation_dataset=None, optimizer=None, epochs=10, batch_size=5):
    x_loss_weight = 0.5
    h_loss_weight = 1 - x_loss_weight

    loss_edges_train, loss_reach_train, loss_parents_train = [], [], []
    loss_edges_val, loss_reach_val, loss_parents_val = [], [], []

    for epoch in range(epochs):
        batch_count = len(train_dataset) // batch_size
        
        cumulated_loss_edges_epoch, cumulated_loss_reach_epoch, cumulated_loss_parents_epoch = 0, 0, 0

        for i in range(batch_count):
            model.train()
            cumulated_loss_edges, cumulated_loss_reach, cumulated_loss_parents = 0, 0, 0
            for j in range(i*batch_size, (i+1)*batch_size):
                graph = train_dataset[j] 
                loss_edges, loss_reach, loss_parents = model(graph)
                loss_edges_output, loss_edges_hints = loss_edges[0], loss_edges[1] # loss for the edges

                cumulated_loss_edges += x_loss_weight * loss_edges_output + h_loss_weight * loss_edges_hints
                cumulated_loss_reach += loss_reach
                cumulated_loss_parents += loss_parents

            cumulated_loss_edges /= batch_size
            cumulated_loss_reach /= batch_size
            cumulated_loss_parents /= batch_size
            
            optimizer.zero_grad()
            cumulated_loss_edges.backward()
            optimizer.step()

            cumulated_loss_edges_epoch += cumulated_loss_edges
            cumulated_loss_reach_epoch += cumulated_loss_reach
            cumulated_loss_parents_epoch += cumulated_loss_parents

        # Convert tensors to lists and append to the respective lists
        loss_edges_train.append(cumulated_loss_edges_epoch.item() / batch_count)
        loss_reach_train.append(cumulated_loss_reach_epoch.item() / batch_count)
        loss_parents_train.append(cumulated_loss_parents_epoch.item() / batch_count)

        if validation_dataset:
            model.eval()
            with torch.no_grad():
                cumulated_loss_edges_val, cumulated_loss_reach_val, cumulated_loss_parents_val = 0, 0, 0
                for k in range(len(validation_dataset)):
                    graph = validation_dataset[k]
                    loss_edges, loss_reach, loss_parents = model(graph)
                    loss_edges_output, loss_edges_hints = loss_edges[0], loss_edges[1] # loss for the edges

                    cumulated_loss_edges_val += x_loss_weight * loss_edges_output + h_loss_weight * loss_edges_hints
                    cumulated_loss_reach_val += loss_reach
                    cumulated_loss_parents_val += loss_parents 

                cumulated_loss_edges_val /= len(validation_dataset)
                cumulated_loss_reach_val /= len(validation_dataset)
                cumulated_loss_parents_val /= len(validation_dataset)

                loss_edges_val.append(cumulated_loss_edges_val.item())
                loss_reach_val.append(cumulated_loss_reach_val.item())
                loss_parents_val.append(cumulated_loss_parents_val.item())

                print(f'Epoch {epoch}, loss_edges {cumulated_loss_edges_epoch.item() / batch_count}, loss_reach {cumulated_loss_reach_epoch.item() / batch_count}, loss_parents {cumulated_loss_parents_epoch.item() / batch_count}, loss_edges_val {cumulated_loss_edges_val.item()}, loss_reach_val {cumulated_loss_reach_val.item()}, loss_parents_val {cumulated_loss_parents_val.item()}')
        
        else:
            print(f'Epoch {epoch}, loss_edges {cumulated_loss_edges_epoch.item() / batch_count}, loss_reach {cumulated_loss_reach_epoch.item() / batch_count}, loss_parents {cumulated_loss_parents_epoch.item() / batch_count}')

    if validation_dataset:
        return loss_edges_train, loss_reach_train, loss_parents_train, loss_edges_val, loss_reach_val, loss_parents_val
    return loss_edges_train, loss_reach_train, loss_parents_train

In [11]:
# split the dataset into training and testing using the train_test_split function
from torch.utils.data import random_split
n=[20, 100]
p=0.3
dataset = RandomGraphDataset(root='./data/medium', gen_num_graph=400, n=n, p=p)
train_dataset, test_dataset = random_split(dataset, [350, 50])

In [ ]:
optimizer = torch.optim.Adam
model = Network()
lr = 0.0002

loss_edges_train, loss_reach_train, loss_parents_train = train(model=model, train_dataset=train_dataset,
      optimizer=optimizer(model.parameters(), lr=lr), epochs=80, batch_size=32)

In [14]:
optimizer = torch.optim.Adam
model = Network()
lr = 0.0002

loss_edges_train, loss_reach_train, loss_parents_train, loss_edges_val, loss_reach_val, loss_parents_val = train(model=model, train_dataset=train_dataset, validation_dataset=test_dataset,
      optimizer=optimizer(model.parameters(), lr=lr), epochs=120, batch_size=32)

Epoch 0, loss_edges 1.0573787689208984, loss_reach 0.0, loss_parents 0.8018753051757812, loss_edges_val 0.8911240100860596, loss_reach_val 0.0833333283662796, loss_parents_val 0.815650463104248
Epoch 1, loss_edges 0.7761232852935791, loss_reach 85.24630737304688, loss_parents 0.7767952919006348, loss_edges_val 0.5715210437774658, loss_reach_val 100.0, loss_parents_val 0.8667091131210327
Epoch 2, loss_edges 0.44747357368469237, loss_reach 100.0, loss_parents 0.873377799987793, loss_edges_val 0.3782954514026642, loss_reach_val 100.0, loss_parents_val 0.900012195110321
Epoch 3, loss_edges 0.39631922245025636, loss_reach 100.0, loss_parents 0.8732841491699219, loss_edges_val 0.3565709590911865, loss_reach_val 100.0, loss_parents_val 0.8675957322120667
Epoch 4, loss_edges 0.35430636405944826, loss_reach 100.0, loss_parents 0.8048425674438476, loss_edges_val 0.3386109471321106, loss_reach_val 100.0, loss_parents_val 0.7866747975349426
Epoch 5, loss_edges 0.3432823896408081, loss_reach 100.0,

In [15]:
torch.save(model.state_dict(), 'model_0002_350_120.pth')

In [22]:
cumulated_loss_edges_test

tensor(0.0081)

In [ ]:
Network().forward(input, dataset[0].edge_index)

In [ ]:
# plot the losses
import matplotlib.pyplot as plt
plt.plot(loss_parents_val, label='val')
plt.plot(loss_edges_train, label='train')

plt.legend()
# use log scale
plt.yscale('log')
plt.show()

In [ ]:
# plot the losses
import matplotlib.pyplot as plt
plt.plot(loss_edges_val, label='val')
plt.plot(loss_edges_train, label='train')

plt.legend()
# use log scale
plt.yscale('log')
plt.show()

In [ ]:
# plot the losses
import matplotlib.pyplot as plt
plt.plot(loss_reach_val, label='val')
plt.plot(loss_reach_train, label='train')

plt.legend()
# use log scale
plt.yscale('log')
plt.show()

In [ ]:
# plot the losses
import matplotlib.pyplot as plt
plt.plot(loss_parents_val, label='train')
plt.plot(loss_reach_val, label='train')
plt.plot(loss_edges_val, label='train')
plt.legend()
# use log scale
plt.yscale('log')
plt.show()

In [ ]:
# plot the losses
import matplotlib.pyplot as plt
plt.plot(loss_edges_train, label='train')
plt.legend()
# use log scale
plt.yscale('log')
plt.show()

In [4]:
from generator import RandomGraphDataset

In [24]:
# split the dataset into training and testing using the train_test_split function
from torch.utils.data import random_split
n=[200, 500]
p=0.3
dataset_test = RandomGraphDataset(root='./data/OOD/200', gen_num_graph=34, n=n, p=p)

In [27]:
dataset_test[0]

CLRSData(edge_index=[2, 27868], pos=[306], length=27868, s=[306], pi=[306], reach_h=[3, 306], pi_h=[3, 306], edges=[27868], edges_h=[3, 27868], hints=[3], inputs=[2], outputs=[1])

In [25]:
x_loss_weight = h_loss_weight = 0.5
model.eval() 

with torch.no_grad():
    cumulated_loss_edges_test, cumulated_loss_reach_test, cumulated_loss_parents_test = 0, 0, 0
    for graph in dataset_test:
        loss_edges, loss_reach, loss_parents = model(graph)
        loss_edges_output, loss_edges_hints = loss_edges[0], loss_edges[1]  # loss for the edges

        cumulated_loss_edges_test += x_loss_weight * loss_edges_output + h_loss_weight * loss_edges_hints
        cumulated_loss_reach_test += loss_reach
        cumulated_loss_parents_test += loss_parents

    cumulated_loss_edges_test /= len(dataset_test)
    cumulated_loss_reach_test /= len(dataset_test)
    cumulated_loss_parents_test /= len(dataset_test)

In [26]:
cumulated_loss_edges_test, cumulated_loss_reach_test, cumulated_loss_parents_test

(tensor(0.0081), tensor(0.), tensor(0.0030))

In [ ]:
dataset.len()

34

In [ ]:
model.eval()